In [ ]:
# Import necessary modules
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
import pickle
from scipy.optimize import curve_fit
import memdiff

%matplotlib inline


# ------------------------ #
#   Function Definitions   #
# ------------------------ #

def genfromxvg(fn):
    ''' most GROMACS xvg files have a header of 8 rows
        and comments starting with @'''
    return np.genfromtxt(fn, skip_header=8, comments='@')

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
# ------------------- #
#   Directory Names   #
# ------------------- #

analysisDir = '../results/martini-bigsimulations/analysis/'
simulation = 'POPC-100CNT-12-8-f11-SNda'
sim_list = ['POPC-100CNT-12-8-f11-SNda-A','POPC-100CNT-12-8-f11-SNda-B','POPC-100CNT-12-8-f11-SNda-C','POPC-100CNT-12-8-f11-SNda-D']

In [ ]:
# ----------------- #
#  Clustering data  #
# ----------------- #

crystals = []
clusters = []
biggest_cluster = []
for simulation in sim_list:
    clusterfile = analysisDir+"clusters/clusters-"+simulation+".dat"
    crystalfile = analysisDir+"crystals/crystals-"+simulation+".dat"
    clusterdata = np.genfromtxt(clusterfile)
    crystaldata = np.genfromtxt(crystalfile)
    time        = crystaldata[:,0]*1e-6
    print( 'Frames:', len(time) )
    crystals.append(crystaldata[:,1])
    clusters.append(clusterdata[:,1])
    biggest_cluster.append(clusterdata[:,2])
crystals        = np.average( np.array(crystals), axis=0 )
clusters        = np.average( np.array(clusters), axis=0 )
biggest_cluster = np.average( np.array(biggest_cluster), axis=0 )

# Fit number of clusters
def fit_nclust(x, b):
    return 99 * np.exp(b * x) + 1
popt_nclust, pcov = curve_fit(fit_nclust, time, clusters)
print(popt_nclust)

# Power Law
def power_law(x, a, b):
    return a*x**b
    
# Fit N_max
popt_nmax, pcov_nmax = curve_fit(power_law, time, biggest_cluster)
print(popt_nmax)
    
# Fit N_average
popt_nav, pcov_nav = curve_fit(power_law, time, 100./clusters)
print(popt_nav)

In [ ]:
# -------------------------- #
#  Radial distribution data  #
# -------------------------- #

filename = analysisDir+"rdf/rdf-"+simulation+".xvg"
rdf_data = genfromxvg(filename)
cnt_dist = rdf_data[:,0]
cnt_rdf  = rdf_data[:,1]

In [ ]:
# ---------------- #
#   Tilting data   #
# ---------------- #

tilt = []
for simulation in sim_list:
    filename = analysisDir+"tilt/tilt-"+simulation+".dat"
    data = np.genfromtxt(filename)
    tilt.append(data[:,1])
    tilt_time = data[:,0]*1e-6
tilt = np.average( np.array(tilt), axis=0 )

In [ ]:
# -------------- #
#   The Figure   #
# -------------- #

fig = plt.figure(figsize=(6.4, 4.8), dpi=300) 
gs = gridspec.GridSpec(2, 2,width_ratios=[1, 1]) 

ax2 = plt.subplot(gs[0,0])
ax0 = plt.subplot(gs[0,1])
ax3 = plt.subplot(gs[1,1])
ax1 = plt.subplot(gs[1,0])

# Plot number of clusters
ax0.plot(time, crystals, lw=3, color='C1', alpha=1, label=r"$r_c = 1.70\,\mathrm{nm}$")
ax0.plot(time, clusters, '-', lw=3, color='C0', alpha=1, label=r"$r_c = 2.25\,\mathrm{nm}$") 
#ax0.plot(time, fit_nclust(time,*popt_nclust), '--', lw=2, color='black', alpha=1, label=r'exp. fit') 
ax0.set(xlabel=r'time$\;\mathrm{[\mu s]}$')
ax0.set(ylabel='#clusters')
ax0.legend(loc='best',frameon=False) #,fontsize=12)
ax0.set_xlim([0,14])
ax0.set_ylim([0,110])
    
# Plot maximum and average size of the clusters  
ax1.plot(time, biggest_cluster, '-', lw=3, color='C0', alpha=0.4, label=r"Maximum cluster size") # $N_\mathrm{max}$
ax1.plot(time[7:], power_law(time[7:],*popt_nmax), '-', lw=2, color='C0', alpha=1) 
ax1.plot(time, 100./clusters, '-', lw=3, color='C2', alpha=0.4, label=r"Average cluster size") # $\bar{N}$
ax1.set(xlabel=r'time$\;\mathrm{[\mu s]}$')
ax1.set(ylabel='cluster size')
ax1.set_xscale("log", nonpositive='clip')
ax1.set_yscale("log", nonpositive='clip')
ax1.legend(loc='upper left',frameon=False) #,fontsize=12)
ax1.set_xlim([0.05,20])
ax1.set_ylim([1,100])
   
## RDF 
ax2.plot(cnt_dist, cnt_rdf, lw=2, label='RDF(CNT-CNT)', color='black')    
ax2.vlines([1.7,2.25],0,210,colors=['C1','C0'],linestyles=['--',':'])
ax2.axis([1., 4, 0, 210])
ax2.set(xlabel=r'$r\;\mathrm{[nm]}$')
ax2.set(ylabel=r'$g(r)$')
ax2.legend(loc='best',frameon=False)  
ax2.annotate( '1.7 nm', xy=[1.06,160], color='C1') # , fontsize=12
ax2.annotate( '2.25 nm', xy=[2.35,100], color='C0') # , fontsize=12

## TILTING
ax3.plot( tilt_time, tilt, lw=2, color='black', alpha=1, label="average tilt")
#ax3.plot(data[10:-9,0]*1e-6, running_mean(data[:,1],20), lw=2, color='black', label="av. tilt angle")
ax3.set_xlim([0,14])
ax3.set_ylim([0.92,1])
ax3.set(xlabel=r'time$\;\mathrm{[\mu s]}$')
ax3.set(ylabel=r'$\left\langle \cos(\theta_\mathrm{tilt}) \right\rangle_\mathrm{CNTs}$')
#ax3.legend(loc='lower right',frameon=False) 
start, end = ax3.get_ylim()
ax3.yaxis.set_ticks(np.arange(start, end+0.01, 0.02))

## LAYOUT AND PRINT
panellabel = ["B","C","A","D"]
for i,axi in enumerate([ax0,ax1,ax2,ax3]):
    #axi.xaxis.label.set_size(16)
    #axi.yaxis.label.set_size(16)
    #axi.tick_params(labelsize=12) 
    #axi.text(-0.1, 1.15, panellabel[i], transform=axi.transAxes, fontsize=16, fontweight='bold', va='top', ha='right')
    axi.text(0.9, 0.25, panellabel[i], transform=axi.transAxes, fontsize=16, va='top', ha='right') # , fontweight='bold'
    
plt.tight_layout()
fig.savefig("figure2.pdf", format='pdf', dpi=300)